In [1]:
from Nb_utils import *

NOTE! Installing ujson may make loading annotations faster.


/home/superorange5/.local/lib/python3.8/site-packages/torchvision/transforms/transforms.py:803: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


In [2]:
from detectron2.evaluation import PascalVOCDetectionEvaluator
from detectron2.evaluation import inference_on_dataset

# 0.Setting

## 0.a skf2c

In [6]:

#---------skf2c-------
model_s2c = load_TSmodel('s2c', 15999)
model_k2c = load_TSmodel('k2c', 19999)
#model_f2c = get_model('f2c', 'final') #8class
model_f2c = load_TSmodel('f2c_1class', 11999)
model_list=[model_s2c,model_k2c,model_f2c]


cfg = setup("configs/pt/final_k2c.yaml")
dataset_name = 'VOC2007_cityval1'



keep_experiments/s2c/cfg.yaml
keep_experiments/s2c/model_0015999.pth
-------- pretrained model loaded ---------
-------- pretrained model loaded ---------
keep_experiments/k2c/cfg.yaml
keep_experiments/k2c/model_0019999.pth
-------- pretrained model loaded ---------
-------- pretrained model loaded ---------
keep_experiments/f2c_1class/cfg.yaml
keep_experiments/f2c_1class/model_0011999.pth
-------- pretrained model loaded ---------
-------- pretrained model loaded ---------


## 0.b ck2bdd

In [5]:
#--------ck2bdd-------
model_c2b = get_model('c2b', 11999)
model_k2b = get_model('k2b', 'final')
model_list=[model_c2b,model_k2b]

cfg = setup("keep_experiments/c2b/cfg.yaml")
dataset_name = 'VOC2007_bddval1' #small'
#dataset_name = 'VOC2007_bddvalsmall'



keep_experiments/c2b/cfg.yaml
keep_experiments/c2b/model_0011999.pth
-------- pretrained model loaded ---------
-------- pretrained model loaded ---------
keep_experiments/k2b/cfg.yaml
keep_experiments/k2b/model_final.pth
-------- pretrained model loaded ---------
-------- pretrained model loaded ---------


## 0.c single model

In [105]:
from FLpkg import FedUtils
def load_sourceonlyModel_nodecorator(cfg_path,model_path):
    cfg = setup_all(cfg_path)   
    
    Trainer =PTrainer_sourceonly
    cfg.defrost()
    cfg.MODEL.WEIGHTS = model_path
    if cfg.FEDSET.DYNAMIC: 
        fedma_model = torch.load(model_path)
        backbone_dim = FedUtils.get_backbone_shape(fedma_model)
        cfg.BACKBONE_DIM = backbone_dim        
    cfg.freeze()
        
    trainer = Trainer(cfg)    
    trainer.resume_or_load(resume=False)
    #model = Trainer.build_model(cfg)
    #DetectionCheckpointer(model).resume_or_load(model_path, resume=False)
    
    return trainer.model

In [29]:
def load_TSmodel_nodecorator(cfg_path, model_path):
    cfg = setup_all(cfg_path)
    #cfg.defrost()
    #cfg.MODEL.WEIGHTS = model_path
    
    Trainer =PTrainer
    model = Trainer.build_model(cfg)
    model_teacher = Trainer.build_model(cfg)
    ensem_ts_model = EnsembleTSModel(model_teacher, model)    
    
    
    DetectionCheckpointer(ensem_ts_model).resume_or_load(model_path, resume=False)
    
    return ensem_ts_model

* set model path & config file

In [169]:

#config_file = "configs/FedMA/ck2b_FedMA.yaml"
config_file = "configs/multi-teacher/ck2b_city_sourceonly_FedMAbackbone-TeacherFedAvg.yaml"
#model_path = "output/FedMA_ck2b_source-only/FedMA_VOC2007_citytrain1_12.pth"
#model_path = "output/FedMA_postFedAvg_skf2c_sourceonly/FedAvg_2.pth"
model_path = "output/multi-teacher_ck2b_city_sourceonly_FedMAbackbone/model_0007999.pth"
#model_to_eval = load_sourceonlyModel("keep_experiments/s2c/cfg.yaml", "./output/FedAvg_skf2c_sourceonly/FedAvg_2.pth")


In [13]:
model_path = "output/multi-teacher_skf2c_foggy_sourceonly/model_final.pth"
config_file = "configs/multi-teacher/skf2c_foggy_sourceonly.yaml"


* load model

In [28]:
model_to_eval = load_TSmodel_nodecorator(config_file,model_path )


-------- pretrained model loaded ---------
-------- pretrained model loaded ---------


Shape of backbone.vgg_block3.0.conv2.bias in checkpoint is torch.Size([262]), while shape of modelStudent.backbone.vgg_block3.0.conv2.bias in model is torch.Size([256]).
backbone.vgg_block3.0.conv2.bias will not be loaded. Please double check and see if this is desired.
Shape of backbone.vgg_block3.0.conv2.weight in checkpoint is torch.Size([262, 256, 3, 3]), while shape of modelStudent.backbone.vgg_block3.0.conv2.weight in model is torch.Size([256, 256, 3, 3]).
backbone.vgg_block3.0.conv2.weight will not be loaded. Please double check and see if this is desired.
Shape of backbone.vgg_block3.0.conv3.weight in checkpoint is torch.Size([256, 262, 3, 3]), while shape of modelStudent.backbone.vgg_block3.0.conv3.weight in model is torch.Size([256, 256, 3, 3]).
backbone.vgg_block3.0.conv3.weight will not be loaded. Please double check and see if this is desired.
Ambiguity found for backbone.vgg_block1.0.conv1.bias in checkpoint!It matches at least two keys in the model (modelTeacher.backbone

ValueError: Cannot match one checkpoint key to multiple keys in the model.

In [ ]:
model_to_eval = load_sourceonlyModel_nodecorator(config_file,model_path )


* load TS model

In [ ]:
model_to_eval =load_TSmodel("keep_experiments/s2c/cfg.yaml", "output/multi-teacher_skf2c_foggy_sourceonly/model_final.pth")

* load FedMA model

In [21]:
from pt.engine.trainer_sourceonly import PTrainer_sourceonly




In [22]:

def get_backbone_shape(fedma_model):
    backbone_shape = []
    for idx,(key,value) in enumerate(fedma_model.items()):
        if idx%2!=0 and idx<26:
            backbone_shape.append(list(value.shape)[0])
    return backbone_shape

In [23]:
def build_fedma_model(model, cfg_path, source_dataset):

    backbone_dim = get_backbone_shape(model)
    print(backbone_dim)

    # build FedMA dynamic backbone model
    cfg = setup_all(cfg_path)
    cfg.defrost()
    cfg.MODEL.WEIGHTS = model_path                
    cfg.DATASETS.TRAIN_LABEL=source_dataset
    #print("current source={}".format(source_dataset))
    cfg.BACKBONE_DIM = backbone_dim    
    cfg.freeze()

    Trainer =PTrainer_sourceonly
    trainer = Trainer(cfg)
    trainer.resume_or_load(resume=False)
    return trainer.model

* load trained model

In [24]:
model_path = 'output/FedMA_skf2c_source-only/VOC2007_kitti1_6/model_final.pth'
fedma_model = torch.load(model_path)
cfg_path = "configs/FedMA/skf2c_FedMA.yaml"
source_dataset = "VOC2007_kitti1"
model_to_eval = build_fedma_model(fedma_model['model'], cfg_path, source_dataset)

[64, 64, 128, 128, 256, 262, 256, 512, 512, 512, 512, 512, 512]


WarmupMultiStepLR is deprecated! Use LRMultipilier with fvcore ParamScheduler instead!


* load ck2b

In [ ]:
model_path = 'output/FedMA_ck2b_source-only/FedMA_VOC2007_citytrain1_7.pth'
fedma_model = torch.load(model_path)
cfg_path = "configs/FedMA/ck2b_FedMA.yaml"
source_dataset = "VOC2007_citytrain1"
model_to_eval = build_fedma_model(fedma_model, cfg_path, source_dataset)

* load skf2c after layer-wise matching

In [4]:
model_path = 'output/FedMA_skf2c_source-only/FedMA_VOC2007_sim1_7.pth'
fedma_model = torch.load(model_path)
cfg_path = "configs/FedMA/skf2c_FedMA.yaml"
source_dataset = "VOC2007_kitti1"
model_to_eval = build_fedma_model(fedma_model, cfg_path, source_dataset)

NameError: name 'build_fedma_model' is not defined

## evaluation

In [7]:
#dataset_name = 'VOC2007_bddval1'
dataset_name = 'VOC2007_cityval1'


In [35]:
#cfg = setup_all(cfg_path)
test_data_loader = build_detection_test_loader(cfg, dataset_name)
evaluator = PascalVOCDetectionEvaluator(dataset_name)
    


In [ ]:
results_i = inference_on_dataset(model_to_eval, test_data_loader, evaluator)

In [38]:
for idx, inputs in enumerate(test_data_loader):
    outputs = model_to_eval(inputs)
    if torch.cuda.is_available():
        torch.cuda.synchronize()
    #print(outputs)
    #evaluator.process(inputs, outputs)
#results = evaluator.evaluate()

TypeError: 'NoneType' object is not subscriptable

In [32]:
results_i

OrderedDict([('bbox',
              {'AP': 27.35157155411934,
               'AP50': 50.97052197740739,
               'AP75': 27.053748762355966})])

In [43]:
dataset_name = 'VOC2007_cityval1'
cfg = setup_all(cfg_path)

In [55]:

def eval_fedma(fedma_model_path,cfg_path, source_dataset, eval_dataset):  
    print(fedma_model_path)
    fedma_model = torch.load(fedma_model_path)
    model_to_eval = build_fedma_model(fedma_model, cfg_path, source_dataset)
    
    cfg = setup_all(cfg_path)
    test_data_loader = build_detection_test_loader(cfg, eval_dataset)
    evaluator = PascalVOCDetectionEvaluator(dataset_name)
    
    results_i = inference_on_dataset(model_to_eval, test_data_loader, evaluator)
    return results_i['bbox']['AP50']

In [61]:
import logging

logging.basicConfig()
logger = logging.getLogger()
logger.setLevel(logging.WARN)

In [62]:

source_dataset_list=["VOC2007_citytrain1","VOC2007_kitti1" ]

cfg_path = "configs/FedMA/ck2b_FedMA.yaml"
eval_dataset = 'VOC2007_bddval1'
for dataset in source_dataset_list:
    print(dataset)
    for i in range(6,13):
        fedma_model_path = 'output/FedMA_ck2b_source-only/FedMA_{}_{}.pth'.format(dataset,i)
        AP50 = eval_fedma(fedma_model_path,cfg_path, dataset, eval_dataset)
        print(AP50)

VOC2007_citytrain1
output/FedMA_ck2b_source-only/FedMA_VOC2007_citytrain1_6.pth
[64, 64, 128, 128, 256, 259, 256, 512, 512, 512, 512, 512, 512]
current source=VOC2007_citytrain1
[64, 64, 128, 128, 256, 259, 256, 512, 512, 512, 512, 512, 512]


backbone.vgg_block3.0.conv2.{bias, weight}
backbone.vgg_block3.0.conv3.{bias, weight}
backbone.vgg_block4.0.conv1.weight


KeyboardInterrupt: 

In [59]:

source_dataset_list=["VOC2007_sim1","VOC2007_kitti1","VOC2007_foggytrain1" ]

cfg_path = "configs/FedMA/skf2c_FedMA.yaml"
eval_dataset = 'VOC2007_cityval1'
for dataset in source_dataset_list:
    print(dataset)
    for i in range(6,13):
        fedma_model_path = 'output/FedMA_skf2c_source-only/FedMA_{}_{}.pth'.format(dataset,i)
        AP50 = eval_fedma(fedma_model_path,cfg_path, dataset, eval_dataset)
        print(AP50)

VOC2007_sim1
output/FedMA_skf2c_source-only/FedMA_VOC2007_sim1_6.pth
[64, 64, 128, 128, 256, 262, 256, 512, 512, 512, 512, 512, 512]
current source=VOC2007_sim1
[64, 64, 128, 128, 256, 262, 256, 512, 512, 512, 512, 512, 512]
0.0006197358761496032
output/FedMA_skf2c_source-only/FedMA_VOC2007_sim1_7.pth
[64, 64, 128, 128, 256, 262, 262, 512, 512, 512, 512, 512, 512]
current source=VOC2007_sim1
[64, 64, 128, 128, 256, 262, 262, 512, 512, 512, 512, 512, 512]
55.73803489085003
output/FedMA_skf2c_source-only/FedMA_VOC2007_sim1_8.pth
[64, 64, 128, 128, 256, 262, 262, 518, 512, 512, 512, 512, 512]
current source=VOC2007_sim1
[64, 64, 128, 128, 256, 262, 262, 518, 512, 512, 512, 512, 512]
0.0
output/FedMA_skf2c_source-only/FedMA_VOC2007_sim1_9.pth
[64, 64, 128, 128, 256, 262, 262, 518, 518, 512, 512, 512, 512]
current source=VOC2007_sim1
[64, 64, 128, 128, 256, 262, 262, 518, 518, 512, 512, 512, 512]
0.00011287548014073345
output/FedMA_skf2c_source-only/FedMA_VOC2007_sim1_10.pth
[64, 64, 128, 1

KeyboardInterrupt: 

## evaluation (defaults.test)

In [20]:
#results = OrderedDict()
test_data_loader = build_detection_test_loader(cfg, dataset_name)
evaluator = PascalVOCDetectionEvaluator(dataset_name)

### generate original car.txt (change code in detectron2/evaluation/pascal_voc_evaluation.py)

In [21]:
#model_c2b.eval()
#results_i = inference_on_dataset(model_k2b.modelStudent, test_data_loader, evaluator)
results_i = inference_on_dataset(model_to_eval, test_data_loader, evaluator)

#tmp file=/tmp/pascal_voc_eval_kzuu496e/{}.txt


In [24]:
results_i

OrderedDict([('bbox',
              {'AP': 10.63140269660908,
               'AP50': 29.176766205840426,
               'AP75': 4.7804266817687475})])

## load car.txt

* print out information
  * self._anno_file_template=data/VOC2007_bddval/Annotations/{}.xml
  * self._image_set_path=data/VOC2007_bddval/ImageSets/Main/val_small.txt
  * self._is_2007=False

In [10]:

_is_2007 = False
res_file_template = "tmp/skf2c_threshold_foggy/{}.txt"

cls_name = 'car'

#------city
_anno_file_template = "data/VOC2007_cityval/Annotations/{}.xml"
_image_set_path = "data/VOC2007_cityval/ImageSets/Main/val.txt"

#------bdd
#_anno_file_template = "data/VOC2007_bddval/Annotations/{}.xml"
#_image_set_path = "data/VOC2007_bddval/ImageSets/Main/val.txt"

In [11]:
from detectron2.evaluation.pascal_voc_evaluation import voc_eval
from collections import OrderedDict, defaultdict

aps = defaultdict(list)

for thresh in range(50, 100, 5):
    rec, prec, ap = voc_eval(
        res_file_template,
        _anno_file_template,
        _image_set_path,
        cls_name,
        ovthresh=thresh / 100.0,
        use_07_metric=_is_2007,
    )
    
    aps[thresh].append(ap * 100)

In [12]:
aps

defaultdict(list,
            {50: [63.90187105980007],
             55: [61.124466035304145],
             60: [58.239042198706706],
             65: [54.56696503077557],
             70: [50.04824192967497],
             75: [44.60209282320927],
             80: [38.23068266882637],
             85: [29.358496451569238],
             90: [17.93286038737506],
             95: [2.0180323047458235]})

In [18]:
model = model_c2b.modelStudent

In [29]:


with torch.no_grad():
    output_result=[]
    for model in model_list_ck2b:
        modelStudent= model.modelStudent
        modelStudent.eval()
        output=modelStudent(data)
        output_result.append(output)
    


In [46]:
output_result[1][0]['instances'].scores


tensor([0.9782, 0.9770, 0.9530, 0.9435, 0.9127, 0.9055, 0.9023, 0.9006, 0.8991,
        0.8486, 0.7825, 0.7520, 0.3332, 0.2876, 0.2741, 0.2580, 0.2209, 0.2179,
        0.1990, 0.1814, 0.1572, 0.1552, 0.1415, 0.1287, 0.0757, 0.0703, 0.0678,
        0.0633, 0.0492, 0.0355, 0.0342, 0.0307, 0.0280, 0.0252, 0.0211, 0.0204,
        0.0202, 0.0162, 0.0151, 0.0113, 0.0109, 0.0101, 0.0090, 0.0088, 0.0078,
        0.0048, 0.0043, 0.0022, 0.0012], device='cuda:0')

In [23]:
evaluator.reset()
evaluator.process(data, output)


In [25]:
evaluator._predictions

defaultdict(list,
            {0: ['b1c66a42-6f7d68ca 0.971 933.1 339.5 1221.0 481.7',
              'b1c66a42-6f7d68ca 0.971 796.0 363.4 905.5 430.0',
              'b1c66a42-6f7d68ca 0.961 751.1 363.6 820.0 413.1',
              'b1c66a42-6f7d68ca 0.946 39.8 351.0 127.8 400.2',
              'b1c66a42-6f7d68ca 0.931 882.3 378.1 966.8 448.7',
              'b1c66a42-6f7d68ca 0.899 247.1 350.0 349.4 397.7',
              'b1c66a42-6f7d68ca 0.896 599.8 355.0 635.8 385.3',
              'b1c66a42-6f7d68ca 0.877 1.0 341.4 58.4 403.1',
              'b1c66a42-6f7d68ca 0.875 733.7 367.5 771.7 404.4',
              'b1c66a42-6f7d68ca 0.756 684.1 360.5 718.3 391.1',
              'b1c66a42-6f7d68ca 0.753 1.0 335.7 10.8 391.8',
              'b1c66a42-6f7d68ca 0.628 204.9 344.2 258.1 386.0',
              'b1c66a42-6f7d68ca 0.594 215.8 359.8 300.0 455.8',
              'b1c66a42-6f7d68ca 0.539 709.3 365.5 743.6 398.0',
              'b1c66a42-6f7d68ca 0.457 692.2 362.0 737.2 395.2',
          

In [20]:
output[0]['instances'].image_size

(720, 1280)

In [101]:
output[0]['instances'].pred_boxes
output[0]['instances'].scores_logists
output[0]['instances'].boxes_sigma
output[0]['instances'].scores

tensor([0.9708, 0.9707, 0.9612, 0.9462, 0.9312, 0.8988, 0.8963, 0.8774, 0.8749,
        0.7561, 0.7531, 0.6275, 0.5939, 0.5390, 0.4565, 0.4178, 0.2566, 0.1998,
        0.1848, 0.1812, 0.1348, 0.1121, 0.0759, 0.0390, 0.0330, 0.0310, 0.0206,
        0.0180, 0.0171, 0.0161, 0.0135, 0.0127, 0.0123, 0.0106, 0.0099, 0.0094],
       device='cuda:0')

In [2]:
len(output[0]['instances'].scores)

NameError: name 'output' is not defined

In [95]:
output[0]['instances'].get_fields

<bound method Instances.get_fields of Instances(num_instances=36, image_height=720, image_width=1280, fields=[pred_boxes: Boxes(tensor([[9.3210e+02, 3.3852e+02, 1.2210e+03, 4.8170e+02],
        [7.9497e+02, 3.6244e+02, 9.0552e+02, 4.3002e+02],
        [7.5011e+02, 3.6257e+02, 8.2003e+02, 4.1306e+02],
        [3.8800e+01, 3.4999e+02, 1.2780e+02, 4.0018e+02],
        [8.8130e+02, 3.7710e+02, 9.6676e+02, 4.4869e+02],
        [2.4614e+02, 3.4897e+02, 3.4941e+02, 3.9769e+02],
        [5.9876e+02, 3.5402e+02, 6.3575e+02, 3.8532e+02],
        [0.0000e+00, 3.4040e+02, 5.8351e+01, 4.0309e+02],
        [7.3273e+02, 3.6651e+02, 7.7169e+02, 4.0439e+02],
        [6.8310e+02, 3.5948e+02, 7.1826e+02, 3.9106e+02],
        [0.0000e+00, 3.3469e+02, 1.0768e+01, 3.9184e+02],
        [2.0393e+02, 3.4324e+02, 2.5814e+02, 3.8604e+02],
        [2.1476e+02, 3.5879e+02, 3.0001e+02, 4.5576e+02],
        [7.0833e+02, 3.6453e+02, 7.4357e+02, 3.9798e+02],
        [6.9122e+02, 3.6096e+02, 7.3717e+02, 3.9521e+02],
  

In [52]:
for idx, inputs in enumerate(test_data_loader):
    output = model_c2b.modelStudent(input)
    print(output)
    

TypeError: 'method' object is not subscriptable

In [63]:
print(next(g))

4


In [51]:
def get_all_inputs_outputs(data_loader,model):
    for data in data_loader:
        yield data, model(data)


evaluator.reset()
for inputs, outputs in get_all_inputs_outputs(test_data_loader,model_c2b.modelTeacher):
      evaluator.process(inputs, outputs)
eval_results = evaluator.evaluate()

TypeError: 'NoneType' object is not subscriptable

In [ ]:
for idx, inputs in enumerate(test_data_loader):

    outputs = model_teacher(inputs)
    evaluator.process(inputs, outputs)
results = evaluator.evaluate()